# Machine Learning

## Setup

In [425]:
import pandas as pd
import numpy as np
import os
import typing
import time
from IPython.display import clear_output

import plotly.express as px
import plotly.graph_objects as go

In [426]:
data = pd.read_csv(r"clean_data.csv")
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,store,weekly_sales,holiday_flag,temperature,fuel_price,cpi,unemployment,holiday_flag_was_missing,temperature_was_missing,fuel_price_was_missing,cpi_was_missing,unemployment_was_missing,year,month,day,dow
0,13.0,2035431.39,0.0,61.11,3.788,130.959226,5.965,False,False,False,False,False,2012,1,6,4
1,17.0,829207.27,0.0,60.07,2.853,126.234600,6.885,False,False,False,False,False,2010,1,10,6
2,10.0,1990371.02,0.0,57.62,3.882,130.645793,7.545,False,False,False,False,False,2012,2,3,4
3,7.0,561145.14,0.0,38.26,2.725,189.704822,8.963,False,False,False,False,False,2010,2,4,3
4,19.0,1549018.68,0.0,66.25,2.958,132.521867,8.099,False,False,False,False,False,2010,2,7,6


## Preprocessing

In [427]:
target = 'weekly_sales'

x = data.drop(target, axis=1)
y = data[[target]]

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num_features = ['temperature', 'fuel_price', 'cpi', 'unemployment']
cat_features = [feature for feature in x.columns if feature not in num_features]

preprocessor = ColumnTransformer([
    ('cat',
     Pipeline([
         ('ohe', OneHotEncoder(drop='first',
                               categories=[sorted(x[col].unique().tolist()) for col in cat_features]))
         ]),
     cat_features),
    ('num',
     Pipeline([
         ('stsc', StandardScaler()),
         ('poly', PolynomialFeatures(degree=2))
         ]),
     num_features),
])

x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [428]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, mean_squared_error, root_mean_squared_error

score_functions = [r2_score, mean_absolute_percentage_error]

def eval_model(model, score_functions=score_functions, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test):
    scores = []
    train_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
    for scorer in score_functions:
        scores.append((
            str(scorer).split(' ')[1],
            scorer(y_train, train_pred),
            scorer(y_test, test_pred)
            ))
        
    return scores

def plot_pred(model, x, y, width=400, height=200, margin=20):
    x = x.copy()
    y = y.copy().iloc[:,0]
    pred = pd.DataFrame(model.predict(x)).iloc[:,0]
    fig = px.scatter(x=y, y=pred, width=width, height=height)
    fig.update_layout(margin=dict(l=margin,r=margin, t=margin, b=margin))
    
    return fig

## Model #1

### Creation

In [429]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

### Evaluation

In [430]:
print(eval_model(model))
plot_pred(model, x_train, y_train).show()
plot_pred(model, x_test, y_test).show()

[('r2_score', 0.9992625164342775, 0.7124354394634478), ('mean_absolute_percentage_error', 0.015970629509145497, 0.3003137675287652)]


### Interpretation

In [431]:
x_train.shape

(90, 86)

We are overfitting, which can be explained by n_samples ≈ n_features (90, 86). Let's try implementing some regularization to our model.

## Model #2 (Regularized)

### Creation

In [432]:
model = Lasso(alpha=10)
model.fit(x_train, y_train)
eval_model(model)

c:\Users\rapha\anaconda3\envs\colab\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:639: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 15559902459.465717, tolerance: 4085869017.774803



[('r2_score', 0.9990388563581358, 0.8287091719902693),
 ('mean_absolute_percentage_error', 0.016342980844531077, 0.24583407425683995)]

Indeed, it does look like even a non-refined regularized model performs significantly better than our non-regularized counterpart. Let's perform a grid search across different regularized models and penalties to find the best performance.

In [433]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet

lasso_grid = {'alpha' : np.logspace(1, 3, 20)}
ridge_grid = {'alpha' : np.logspace(-3, -1, 20)}
elastic_grid = {'alpha' : np.logspace(-6, -4, 10),
                'l1_ratio' : np.arange(0, 1, 0.1)}

models = [Lasso, Ridge, ElasticNet]
search_results = []

for model in models:
    if model == Lasso:
        param_grid = lasso_grid
    elif model == Ridge:
        param_grid = ridge_grid
    else:
        param_grid = elastic_grid
        
    model = model(max_iter=10_000)
    search = GridSearchCV(model, param_grid, cv=5)
    search.fit(x_train, y_train)
    search_results.append((model, search.best_params_, search.score(x_test, y_test)))
    
clear_output()
search_results

[(Lasso(max_iter=10000), {'alpha': 183.29807108324357}, 0.9310098972861118),
 (Ridge(max_iter=10000), {'alpha': 0.06158482110660261}, 0.8960169218671297),
 (ElasticNet(max_iter=10000),
  {'alpha': 1.2915496650148827e-05, 'l1_ratio': 0.4},
  0.7666989701317438)]

Our best model is an L1 regularized linear regression (Lasso) with a regularization coefficient of ~200 and a test result of ~0.933. Let's retrain this model from scratch to confirm its performance.

In [434]:
model = Lasso(alpha=200)
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.9322178664265564

### Evaluation

In [435]:
print(eval_model(model))
plot_pred(model, x_train, y_train).show()
plot_pred(model, x_test, y_test).show()

[('r2_score', 0.9970968515685101, 0.9322178664265564), ('mean_absolute_percentage_error', 0.03096307038148441, 0.13878041407082997)]


In [436]:
# Create feature importance dataframe
feature_importance = pd.DataFrame({
    'feat' : preprocessor.get_feature_names_out(),
    'coef' : model.coef_
})

# Separate numerical and categorical feature importances
num_importance = feature_importance[feature_importance['feat'].str.contains('num')]
num_importance.loc[:,'coef'] = num_importance['coef'].abs()
cat_importance = feature_importance[feature_importance['feat'].str.contains('cat')]

# Group features of same category together, find mean absolute importance
cat_importance.loc[:,'feat'] = cat_importance['feat'].str[:8]
cat_importance.loc[:,'coef'] = cat_importance['coef'].abs()
cat_importance = cat_importance.groupby('feat').mean().reset_index()

feature_importance = pd.concat([num_importance, cat_importance])

with pd.option_context('display.max_rows', None):
    display(
        feature_importance.sort_values(by='coef', ascending=False)
    )

,feat,coef
6,cat__sto,617668.290622
0,cat__cpi,215093.784343
3,cat__fue,180540.614538
4,cat__hol,160650.655417
1,cat__day,69395.539098
7,cat__tem,48491.608103
8,cat__une,45526.602718
5,cat__mon,45293.887489
75,num__unemployment,43956.723238
80,num__fuel_price^2,40571.862006


### Interpretation

We are still overfitting slightly, but much less thanks to model regularization.

# Conclusion

It looks like our model is relying heavily on the identifier of a walmart store, rather than any economic factors we have access to. We can suppose that having access to location information about stores, notably things such as store size and median income around store then we could hope for significantly improved results.